In [ ]:
import pandas as pd

In [ ]:
df =pd.read_csv("airports.csv")
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
unique_types = df["type"].unique()

In [ ]:
filtered_airports = df[df["type"].isin(["medium_airport", "large_airport"])]
filtered_airports.head()

In [ ]:
search_result = df[df.applymap(lambda x: 'LAX' in str(x)).any(axis=1)]
print(search_result)

In [ ]:
row_2214 = df[df['id'] == 2214]

# Display the row
print(row_2214)

In [ ]:
iata = df[df['iata_code'] == "JFK"]

# Display the row
print(iata)

In [ ]:
data = {'origin': ["HAM"],
        'destination': ["LAX"]}

df = pd.DataFrame(data)

# Write the DataFrame to a CSV file
df.to_csv('output.csv', index=False)

print("CSV file 'output.csv' has been created successfully.")

In [ ]:
own=pd.read_csv("output.csv")
print(own.head())

In [ ]:
import pandas as pd

# Sample data
# airport_data = {
#     'code': ['LAX', 'JFK', 'SFO'],
#     'latitude_deg': [33.9425, 40.6413, 37.6217],
#     'longitude_deg': [-118.4081, -73.7781, -122.3755]
# }

# own_data = {
#     'origin': ['LAX', 'JFK', 'SFO'],
#     'destination': ['JFK', 'SFO', 'LAX']
# }

# Create DataFrames
airport_df = pd.read_csv("airports.csv")
own_df = pd.read_csv("output.csv")

def get_coordinates(airport_code):
    # Get latitude and longitude for given airport code
    row = airport_df[airport_df['iata_code'] == airport_code].iloc[0]
    return row['latitude_deg'], row['longitude_deg']

def get_flight_coordinates(row):
    # Extract airport codes from 'origin' and 'destination' columns
    origin_code = row['origin']
    destination_code = row['destination']
    
    # Get coordinates for origin and destination airports
    origin_lat, origin_lng = get_coordinates(origin_code)
    destination_lat, destination_lng = get_coordinates(destination_code)
    
    return origin_lat, origin_lng, destination_lat, destination_lng

# Apply the function to each row in own_df
own_df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = own_df.apply(get_flight_coordinates, axis=1, result_type='expand')

print(own_df)


In [15]:
#from here
import pandas as pd

airport_df = pd.read_csv("data/airports.csv")
own_df = pd.read_csv("data/own.csv")

def get_coordinates(airport_code):
    # Get latitude and longitude for given airport code
    row = airport_df[airport_df['iata_code'] == airport_code].iloc[0]
    return row['latitude_deg'], row['longitude_deg']

def get_flight_coordinates(row):
    # Extract airport codes from 'origin' and 'destination' columns
    origin_code = row['origin']
    destination_code = row['destination']
    
    # Get coordinates for origin and destination airports
    origin_lat = row['origin_lat'] if pd.notnull(row['origin_lat']) else get_coordinates(origin_code)[0]
    origin_lng = row['origin_lng'] if pd.notnull(row['origin_lng']) else get_coordinates(origin_code)[1]
    destination_lat = row['destination_lat'] if pd.notnull(row['destination_lat']) else get_coordinates(destination_code)[0]
    destination_lng = row['destination_lng'] if pd.notnull(row['destination_lng']) else get_coordinates(destination_code)[1]
    
    return origin_lat, origin_lng, destination_lat, destination_lng

# Apply the function to each row in own_df
own_df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = own_df.apply(get_flight_coordinates, axis=1, result_type='expand')

print(own_df)


  origin destination  origin_lat  origin_lng  destination_lat  \
0    HAM         LAX   53.630402     9.98823        33.942501   

   destination_lng     distance  more_info  
0      -118.407997  9086.586271        NaN  


In [16]:
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the great-circle distance between two points on the Earth."""
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of Earth in kilometers
    return c * r

# Iterate over rows
for index, row in own_df.iterrows():
    if pd.isnull(row['distance']):
        lat1, lon1, lat2, lon2 = row['origin_lat'], row['origin_lng'], row['destination_lat'], row['destination_lng']
        if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
            own_df.at[index, 'distance'] = calculate_distance(lat1, lon1, lat2, lon2)

print(own_df)
own_df.to_csv('output.csv', index=False)

  origin destination  origin_lat  origin_lng  destination_lat  \
0    HAM         LAX   53.630402     9.98823        33.942501   

   destination_lng     distance  more_info  
0      -118.407997  9086.586271        NaN  


In [17]:
import pandas as pd

def add_row_to_dataframe(df, values_dict):
    # Create a new DataFrame with the values from the dictionary
    new_row = pd.DataFrame([values_dict])

    # Append the new row to the original DataFrame
    df = pd.concat([df, new_row], ignore_index=True)

    return df

# Example usage:
# Suppose you have an existing DataFrame
#df = pd.DataFrame(columns=['A', 'B', 'C'])

# Dictionary containing values for the new row
new_row_values = {'origin': "HAM", 'destination': "JFK",}

# Add the new row to the DataFrame
own_df = add_row_to_dataframe(own_df, new_row_values)
# df.to_csv('output.csv', index=False)
print(own_df)


  origin destination  origin_lat  origin_lng  destination_lat  \
0    HAM         LAX   53.630402     9.98823        33.942501   
1    HAM         JFK         NaN         NaN              NaN   

   destination_lng     distance  more_info  
0      -118.407997  9086.586271        NaN  
1              NaN          NaN        NaN  


In [20]:
def get_column_sum(df, column_name):
    column_sum=df[column_name].sum()
    return column_sum
sum_kilo=get_column_sum(own_df,'distance')
print(sum_kilo)

9086.586271240469


In [22]:
def surrounded_world(distance):
    surrounded=distance/40030
    return surrounded

surrounden=surrounded_world(40000)
print(surrounden)


0.9992505620784412


In [23]:
df = pd.read_csv("data/own.csv")
def most_common_value(df, column_name):
    """Find the most common value in a column and its frequency."""
    # Count the occurrences of each value in the column
    value_counts = df[column_name].value_counts()
    
    # Find the value that occurs the most
    most_common_value = value_counts.idxmax()
    
    # Find the frequency of the most common value
    frequency = value_counts.max()
    
    return most_common_value, frequency
most_common, frequency = most_common_value(df, "origin")
print(f"The most common value in column 'A' is {most_common}, which occurs {frequency} times.")

The most common value in column 'A' is HAM, which occurs 2 times.
